## Information

Maria is a 25-year-old US Army veteran, newly returned to the civilian workforce. She has recently completed a six-year commitment with the Army. During her time in the Army, she worked in supply management and logistics. She has decided to pursue a degree in Management Systems and Information Technology.

Maria has asked you to use your data skills to help her search for the best school for her. She is willing to relocate anywhere in the continental United States, but she has a few criteria that her ideal schools must satisfy: 

Safety of the city
Schools should be offering a degree in IT
Ranking of the school

## Task
Clean the Data for
Any duplicates

Missing Values

Inconsistent values

Data Enrichment
Calculate School Ranking
Calculate the overall crime rate
Structure the Data: Merge the tables and produce the dataset which must have:
Top 5 schools on rankings for It college

Be in a city that is below 50th percentile in overall crime

Remove unnecessary columns

In [231]:
#library import, data loading

import pandas as pd
import numpy as np
import re

#dataset with colledge information
collegeDf = pd.read_excel('CollegeScorecard2.xlsx')
print(collegeDf.head())

#dataset with information about crime rate
crimeRateDf = pd.read_excel('Crime_2015.xlsx')
print(crimeRateDf.head())

     uid  RANK                                   UNIVERSITY             CITY  \
0  10023     5                           Cornell University           Ithaca   
1  10034     6                     University of Washington      Seattle ...   
2  10294     7  Columbia University in the City of New York    New York City   
3  10388    10          University of Minnesota-Twin Cities  Minneapolis ...   
4  10524    12                   University of Pennsylvania     Philadelphia   

         Courses  AcademicScore  Staff/TeacherRatio  Citation Index  \
0  IT/Management            7.0                 8.0             5.0   
1             IT            9.0                 8.0             9.0   
2     Management            6.0                 7.0             7.0   
3        Medical            5.0                 6.0             7.0   
4   Architecture            6.0                 4.0             6.0   

                                              NPCURL  
0     galileo.aamu.edu/netpricecalcul

In [232]:
#preliminary information about datasets
print(f"crimedf shape: {crimeRateDf.shape}")
print(f"crimedf information: {crimeRateDf.info()}")
print(f"crimedf describe: {crimeRateDf.describe}")

crimedf shape: (37, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CITY               37 non-null     object 
 1   ViolentCrime       37 non-null     float64
 2   Murder             37 non-null     float64
 3   Robbery            37 non-null     float64
 4   PropertyCrime      36 non-null     float64
 5   Burglary           37 non-null     float64
 6   Theft              36 non-null     float64
 7   MotorVehicleTheft  37 non-null     float64
dtypes: float64(7), object(1)
memory usage: 2.4+ KB
crimedf information: None
crimedf describe: <bound method NDFrame.describe of                  CITY  ViolentCrime  Murder  Robbery  PropertyCrime  Burglary  \
0              Ithaca        1160.0    15.1    122.2         4701.9    1179.5   
1         Seattle ...        1070.1     7.6    126.6         4233.9     801.2   
2       New York City 

In [233]:
print(f"colledgedf shape: {collegeDf.shape}")
print(f"colledgedf information: {collegeDf.info()}")
print(f"colledgedf describe: {collegeDf.describe}")

colledgedf shape: (31, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uid                 31 non-null     int64  
 1   RANK                31 non-null     int64  
 2   UNIVERSITY          31 non-null     object 
 3   CITY                31 non-null     object 
 4   Courses             30 non-null     object 
 5   AcademicScore       22 non-null     float64
 6   Staff/TeacherRatio  22 non-null     float64
 7   Citation Index      20 non-null     float64
 8   NPCURL              31 non-null     object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.3+ KB
colledgedf information: None
colledgedf describe: <bound method NDFrame.describe of       uid  RANK                                         UNIVERSITY  \
0   10023     5                                 Cornell University   
1   10034     6                           Unive

As in "collegde" data set there is a need only for Rank, City, University name and Courses information, unnessesary columns is better to remove at the first stage in order to improve performance and use computational resources rationaly. 

Additionaly, as there is no missing values in the columns of interest, there is no need to work on it. The one row in Courses that has NA will be removed when filtering dataset for leaving universities with the IT course only. If it would be a bigger data set - it would be nessesary to drop rows with missing values first.  

In [234]:
collegeDf.columns
# drop columns that are not needed for the analysis
columns_to_drop = ['uid','AcademicScore', 'Staff/TeacherRatio', 'Citation Index', 'NPCURL']
colledgeDf_cleaned = collegeDf.drop(columns =columns_to_drop )
colledgeDf_cleaned.columns

Index(['RANK', 'UNIVERSITY', 'CITY', 'Courses'], dtype='object')

In [235]:
# cleaning CITY column from unnsesary symbols such as dots
# Function to clean the string
def clean_string(s):
    if isinstance(s, str):
        # Replace periods with spaces
        s = s.replace('.', ' ')
        # Remove tabs, newlines, and other whitespace characters
        s = re.sub(r'\s+', ' ', s)  # Replace multiple spaces, tabs, or newlines with a single space
        # Strip leading and trailing spaces
        s = s.strip()
    return s 

colledgeDf_cleaned['CITY'] = colledgeDf_cleaned['CITY'].apply(clean_string)
print(colledgeDf_cleaned['CITY'])


0             Ithaca
1            Seattle
2      New York City
3        Minneapolis
4       Philadelphia
5        Los Angeles
6          New Haven
7            Madison
8      New York City
9          Princeton
10       Los Angeles
11        Pittsburgh
12      East Lansing
13         San Diego
14       Gainesville
15            Irvine
16     North Chicago
17            Durham
18          Evanston
19     New Brunswick
20    Salt Lake City
21           Raleigh
22        Pittsburgh
23          Pasadena
24     Santa Barbara
25           Lincoln
26           Medford
27            Eugene
28             Provo
29         Iowa City
30           Pullman
Name: CITY, dtype: object


In [236]:
#removing duplicates
#check for duplicates in UNIVERSITY column
unique_names = colledgeDf_cleaned['UNIVERSITY'].unique()
print(f"number of unique universities: {len(unique_names)}")
print(f"number of dataset rows: {colledgeDf_cleaned.shape }") 

if(len(unique_names) == colledgeDf_cleaned.shape[0]):
    print("There is no duplicates in University columns")
else:
    print(f"There are {colledgeDf_cleaned.shape[0] - len(unique_names)} duplicates in the University column")


number of unique universities: 31
number of dataset rows: (31, 4)
There is no duplicates in University columns


In [237]:
#adding a column that indicates if there is a degree in IT offered
def isItDegree(s):
    if isinstance(s, str):
        if 'IT' in s:
            s = True
        else:
            s = False
        
    return s

colledgeDf_cleaned['IsITDegree'] = colledgeDf_cleaned['Courses'].apply(lambda x: pd.Series(isItDegree(x)))
print(colledgeDf_cleaned.head())

   RANK                                   UNIVERSITY           CITY  \
0     5                           Cornell University         Ithaca   
1     6                     University of Washington        Seattle   
2     7  Columbia University in the City of New York  New York City   
3    10          University of Minnesota-Twin Cities    Minneapolis   
4    12                   University of Pennsylvania   Philadelphia   

         Courses  IsITDegree  
0  IT/Management         1.0  
1             IT         1.0  
2     Management         0.0  
3        Medical         0.0  
4   Architecture         0.0  


In [238]:
#Keep only those Universities that have degree in IT sorted by RANK
colledgeDf_cleaned= colledgeDf_cleaned[colledgeDf_cleaned['IsITDegree'] == True]
colledgeDf_cleaned = colledgeDf_cleaned.sort_values(by="RANK", ascending=False)
print(colledgeDf_cleaned)

    RANK                                         UNIVERSITY           CITY  \
16  1083  Rosalind Franklin University of Medicine and S...  North Chicago   
21    42                    North Carolina State University        Raleigh   
17    31                                    Duke University         Durham   
15    29                   University of California, Irvine         Irvine   
13    24                University of California, San Diego      San Diego   
10    20                  University of Southern California    Los Angeles   
8     18                                New York University  New York City   
5     13              University of California, Los Angeles    Los Angeles   
1      6                           University of Washington        Seattle   
0      5                                 Cornell University         Ithaca   

                      Courses  IsITDegree  
16             ITArchitecture         1.0  
21              IT/management         1.0  
17  Elect

Cleaning crime rate dataset 
Step 1 clean the CITY column
Step 2 drop rows with missing values as dataset has only one such row. In case of big amount of missing values techniques for replacing missing values would apply, for instance replacing missing value with a mode ar mean value. In the more difficult case - it might be nessesary to replace missing value with the nearest neighbor known value. 

In [239]:
#cleaning of the CITY column
crimeRateDf['CITY'] = crimeRateDf['CITY'].apply(clean_string)
print(crimeRateDf['CITY'])

#drop rows with missinf values
crimeRateDf = crimeRateDf.dropna()

0             Ithaca
1            Seattle
2      New York City
3        Minneapolis
4       Philadelphia
5        Los Angeles
6      North Chicago
7            Madison
8      New York City
9          Princeton
10       Los Angeles
11        Pittsburgh
12      East Lansing
13         San Diego
14       Gainesville
15            Irvine
16     North Chicago
17            Durham
18          Evanston
19     New Brunswick
20    Salt Lake City
21           Raleigh
22        Pittsburgh
23          Pasadena
24     Santa Barbara
25           Lincoln
26           Medford
27            Eugene
28             Provo
29         Iowa City
30           Pullman
31         Rochester
32           Fairfax
33        Providence
34         Nashville
35           Hanover
36            Newark
Name: CITY, dtype: object


In [240]:
#check for duplicates 
unique_cities= crimeRateDf['CITY'].unique()
print(f"number of unique cities: {len(unique_cities)}")
print(f"number of dataset rows: {crimeRateDf.shape[0] }") 

if(len(unique_cities) == colledgeDf_cleaned.shape[0]):
    print("There is no duplicates in CITY columns")
else:
    print(f"There are {crimeRateDf.shape[0] - len(unique_cities)} duplicates in the CITY column")


number of unique cities: 33
number of dataset rows: 36
There are 3 duplicates in the CITY column


In [241]:
#drop duplicates
crimeRateDf=crimeRateDf.drop_duplicates(subset=["CITY"])
print(crimeRateDf.shape)

(33, 8)


## Calculation of the total crime rate and define cities that are below 50th percentile in overall crime

In [242]:
#calculation of the total crime rate by addint totalCrime column 
crimeRateDf["totalCrimeRate"] = crimeRateDf["ViolentCrime"] + crimeRateDf["Murder"] + crimeRateDf["Robbery"] + crimeRateDf["PropertyCrime"] + crimeRateDf["Burglary"] + crimeRateDf["Theft"] + crimeRateDf["MotorVehicleTheft"]

#sort cities in Dewcending order
crimeRateDf = crimeRateDf.sort_values(by="totalCrimeRate", ascending=True)
columnsToPrint=["CITY", "totalCrimeRate"]
print(crimeRateDf[columnsToPrint])

#drop columns that are not needed for the analysis
crimeRateDf = crimeRateDf.drop(columns=["ViolentCrime", "Murder", "Robbery", "PropertyCrime", "Burglary", "Theft","MotorVehicleTheft"])
crimeRateDf.shape

              CITY  totalCrimeRate
33      Providence          4644.5
34       Nashville          4685.4
29       Iowa City          5387.4
35         Hanover          5390.7
17          Durham          5907.4
23        Pasadena          6150.4
30         Pullman          6286.1
5      Los Angeles          6298.2
32         Fairfax          6484.4
6    North Chicago          6730.4
26         Medford          6779.1
28           Provo          6856.7
11      Pittsburgh          6988.4
12    East Lansing          7007.4
3      Minneapolis          7039.5
31       Rochester          7245.1
19   New Brunswick          7447.0
27          Eugene          7453.4
25         Lincoln          7559.1
24   Santa Barbara          7572.4
15          Irvine          7654.9
14     Gainesville          7684.8
36          Newark          7818.3
20  Salt Lake City          7910.7
7          Madison          7951.4
21         Raleigh          8121.4
18        Evanston          8156.9
9        Princeton  

(33, 2)

In [243]:
#define cities where crime is below 50 percentile
def pct50(column):
    return column.quantile(0.3)
Below50pctMark = crimeRateDf["totalCrimeRate"].agg(pct50)
print(Below50pctMark)
cityToLive = crimeRateDf[crimeRateDf["totalCrimeRate"]<Below50pctMark]
print(cityToLive)


6759.620000000001
             CITY  totalCrimeRate
33     Providence          4644.5
34      Nashville          4685.4
29      Iowa City          5387.4
35        Hanover          5390.7
17         Durham          5907.4
23       Pasadena          6150.4
30        Pullman          6286.1
5     Los Angeles          6298.2
32        Fairfax          6484.4
6   North Chicago          6730.4


## Defining best schools for Maria

In [244]:
#merge datasets 
topSafeColledge = pd.merge(cityToLive, colledgeDf_cleaned, on='CITY', how='inner')

topSafeColledge = topSafeColledge.sort_values(by="RANK", ascending=False)

topSafeColledge


,CITY,totalCrimeRate,RANK,UNIVERSITY,Courses,IsITDegree
3,North Chicago,6730.4,1083,Rosalind Franklin University of Medicine and S...,ITArchitecture,1.0
0,Durham,5907.4,31,Duke University,Electrnoics/IT/Biomedical,1.0
1,Los Angeles,6298.2,20,University of Southern California,IT/Medical/Arts0,1.0
2,Los Angeles,6298.2,13,"University of California, Los Angeles",Architecture/ARTS/IT,1.0


### Conclusion: there are only 4 univercities that offer an IT degree and are in cities with the crime rate below 50 percentile, the most popular city is Los Angeles as it has two universities that fullfil requirements